# Loading your datasets and fine-tuning models

In [ ]:
! pip --quiet install transformers datasets evaluate rouge_score torch bert_score py7zr

Log into hugging face to access private datasets and to push trained models

In [ ]:
from huggingface_hub import login, logout

login("<access token>", add_to_git_credential=True)

Load necessary datasets

In [ ]:
from datasets import load_dataset

xsum = load_dataset("xsum")
billsum = load_dataset("billsum")
samsum = load_dataset("samsum")

# private repos
old_books_data_train = load_dataset("psin/old_books_data_train")
old_books_data_test = load_dataset("psin/old_books_data_test")
new_books_data_train = load_dataset("psin/new_books_data_train")
new_books_data_test = load_dataset("psin/new_books_data_test")

Print the dataset to see how texts and summaries are tagged. Remove any irrelevant columns



In [ ]:
print(xsum)
print(billsum)
print(samsum)
print(old_books_data_train)

Removes extraneous columsn

In [ ]:
xsum = xsum.remove_columns(['id'])
billsum = billsum.remove_columns(['title'])
samsum.remove_columns('id')

Get pre-trained T5 tokenizer

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Define functions for pre-processing the data. Pre-pend the texts with the "summarize: " prefix.

In [ ]:
def preprocess_function_fiction(examples, text_column_name="text", summary_column_name="summary", prefix = "summarize: "):
    # already prefixed summary tag
    inputs = [doc for doc in examples[text_column_name]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples[summary_column_name], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def preprocess_function_data(examples, text_column_name="text", summary_column_name="summary", prefix = "summarize: "):
    inputs = [prefix + doc for doc in examples[text_column_name]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples[summary_column_name], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Apply the pre-processing function to the data 

In [ ]:
tokenized_billsum = billsum.map(preprocess_function_billsum, batched=True)

Load the relevant evaluation metrics

In [ ]:
import evaluate
import numpy as np
from datasets import load_metric

rouge = evaluate.load("rouge")
bertscore = load_metric("bertscore")

Function to compute metrics and return them as a dictionary. Average values over the batch.

In [ ]:
def compute_metrics(eval_pred, decoded=False):
    predictions, labels = eval_pred
    if decoded:
       decoded_preds = predictions
       decoded_labels = labels
    else:
      decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
      labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
      decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    
    # paper reccomended rescaling baseline 
    bert_result = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en", rescale_with_baseline=True)
    bert_result_final = bert_result
    print(bert_result.keys())
    bert_result_final['precision'] = np.round(np.mean(bert_result['precision']), 4)
    bert_result_final['recall'] = np.round(np.mean(bert_result['recall']), 4)
    bert_result_final['f1'] = np.round(np.mean(bert_result['f1']))

    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    rouge_result["gen_len"] = np.mean(prediction_lens)
    rouge_result_final = {k: round(v, 4) for k, v in rouge_result.items()}
    
    
    
    final_result = rouge_result_final | bert_result_final
    return final_result

Load the base model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Load the data collator

In [ ]:
from transformers import DataCollatorForSeq2Seq
checkpoint = "t5-small"
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

Train the model

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="model name",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    seed = 72,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum['train'],
    eval_dataset=tokenized_billsum['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

If pushing large files

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!git lfs install

Push the model to your account

In [ ]:
trainer.push_to_hub()
